In [2]:
from fitparse import FitFile
from datetime import datetime

stages = FitFile('./data/stages/stages.fit')
kickr = FitFile('./data/stages/kickr.fit')

In [4]:
list(stages.get_messages())

[<DataMessage: file_id (#0) -- local mesg: #0, fields: [time_created: 2020-05-31 17:37:54, type: activity, manufacturer: wahoo_fitness, product: 31, serial_number: 3578357760]>,
 <DataMessage: developer_data_id (#207) -- local mesg: #0, fields: [developer_data_index: 0, manufacturer_id: wahoo_fitness]>,
 <DataMessage: developer_data_id (#207) -- local mesg: #0, fields: [developer_data_index: 1, manufacturer_id: wahoo_fitness]>,
 <DataMessage: developer_data_id (#207) -- local mesg: #0, fields: [developer_data_index: 5, manufacturer_id: wahoo_fitness]>,
 <DataMessage: developer_data_id (#207) -- local mesg: #0, fields: [developer_data_index: 2, manufacturer_id: wahoo_fitness]>,
 <DataMessage: developer_data_id (#207) -- local mesg: #0, fields: [developer_data_index: 6, manufacturer_id: wahoo_fitness]>,
 <DataMessage: developer_data_id (#207) -- local mesg: #0, fields: [developer_data_index: 3, manufacturer_id: wahoo_fitness]>,
 <DataMessage: developer_data_id (#207) -- local mesg: #0, f

In [9]:
def get_time_power(file):
    time_power = []
    for data_message in file.get_messages():
        vals = data_message.get_values()
        if vals.get('timestamp') is not None and vals.get('power') is not None:
            time_power += [(vals['timestamp'], vals['power'])]
    return time_power

kickr_time_power = get_time_power(kickr)
stages_time_power = get_time_power(stages)

In [12]:
import itertools

data_format = [{
    'datetime': datetime(2020, 4, 24, 13, 39, 1),
    'stages': 100,
    'kickr': 100
}]

idx_stages = 0
idx_kickr = 0

stages_last_power = 0
kickr_last_power = 0

bundled_time_power = []

while idx_stages < len(stages_time_power) and idx_kickr < len(kickr_time_power):
    stages_time = stages_time_power[idx_stages][0]
    stages_power = stages_time_power[idx_stages][1]
    
    kickr_time = kickr_time_power[idx_kickr][0]
    kickr_power = kickr_time_power[idx_kickr][1]
    
    if stages_time == kickr_time:
        bundled_time_power += [{
                'datetime': stages_time,
                'stages': stages_power,
                'kickr': kickr_power
        }]
        stages_last_power = stages_power
        kickr_last_power = kickr_power
        idx_kickr += 1
        idx_stages += 1
    elif stages_time < kickr_time:
        bundled_time_power += [{
                'datetime': stages_time,
                'stages': stages_power,
                'kickr': kickr_last_power
        }]
        stages_last_power = stages_power
        idx_stages += 1
    else:
        bundled_time_power += [{
                'datetime': kickr_time,
                'stages': stages_last_power,
                'kickr': kickr_power
        }]
        kickr_last_power = kickr_power
        idx_kickr += 1

bundled_time_power

[{'datetime': datetime.datetime(2020, 5, 31, 17, 37, 49),
  'stages': 0,
  'kickr': 168},
 {'datetime': datetime.datetime(2020, 5, 31, 17, 37, 50),
  'stages': 0,
  'kickr': 122},
 {'datetime': datetime.datetime(2020, 5, 31, 17, 37, 51),
  'stages': 0,
  'kickr': 80},
 {'datetime': datetime.datetime(2020, 5, 31, 17, 37, 52),
  'stages': 0,
  'kickr': 80},
 {'datetime': datetime.datetime(2020, 5, 31, 17, 37, 53),
  'stages': 0,
  'kickr': 80},
 {'datetime': datetime.datetime(2020, 5, 31, 17, 37, 54),
  'stages': 0,
  'kickr': 80},
 {'datetime': datetime.datetime(2020, 5, 31, 17, 37, 55),
  'stages': 0,
  'kickr': 81},
 {'datetime': datetime.datetime(2020, 5, 31, 17, 37, 56),
  'stages': 0,
  'kickr': 83},
 {'datetime': datetime.datetime(2020, 5, 31, 17, 37, 57),
  'stages': 0,
  'kickr': 173},
 {'datetime': datetime.datetime(2020, 5, 31, 17, 37, 58),
  'stages': 220,
  'kickr': 180},
 {'datetime': datetime.datetime(2020, 5, 31, 17, 37, 59),
  'stages': 212,
  'kickr': 176},
 {'datetime'

In [13]:
from statistics import mean 

stages_power = [x['stages'] for x in bundled_time_power]
kickr_power = [x['kickr'] for x in bundled_time_power]

len(stages_power), len(kickr_power), mean(stages_power), mean(kickr_power)

(4164, 4164, 244.39937560038425, 248.4615754082613)

In [57]:
import plotly.express as px
import plotly.graph_objects as go

import pandas as pd

df = pd.DataFrame(bundled_time_power)
fig = px.line(df, x='datetime', y=['kickr', 'stages'])
fig.show()

ValueError: Value of 'x' is not the name of a column in 'data_frame'. Expected one of ['stages', 'kickr'] but received: datetime

In [56]:
START = '17:47:28'
END = '17:55:01'

import datetime

def dt(h, m, s):
    return datetime.datetime(2020, 5, 31, h, m, s)

start_dt = dt(int(START.split(':')[0]),int(START.split(':')[1]),int(START.split(':')[2]))
end_dt = dt(int(END.split(':')[0]),int(END.split(':')[1]),int(END.split(':')[2]))

# df[df['datetime'] > start_dt and df['datetime'] < end_dt]
df = df.set_index('datetime')
df.loc[start_dt:end_dt]


,stages,kickr
datetime,,
2020-05-31 17:47:28,213,250
2020-05-31 17:47:29,250,250
2020-05-31 17:47:30,244,249
2020-05-31 17:47:31,238,250
2020-05-31 17:47:32,236,250
...,...,...
2020-05-31 17:54:57,272,299
2020-05-31 17:54:58,278,297
2020-05-31 17:54:59,285,296
